In [2]:
import torch
from peft import LoraConfig
from trl import SFTConfig

# Dictionnaire des hyperparamètres (utile pour le logging plus tard)
hyperparameters = {
    "lora_r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    "learning_rate": 2.0e-4,
    "batch_size": 1,
    "num_train_epochs": 1,
}

# Configuration LoRA
lora_config = LoraConfig(
    r=hyperparameters["lora_r"],
    lora_alpha=hyperparameters["lora_alpha"],
    lora_dropout=hyperparameters["lora_dropout"],
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)

# Configuration de l'entraînement SFT (Supervised Fine-Tuning)
sft_config = SFTConfig(
    output_dir="./results", # Dossier où les checkpoints du modèle seront sauvegardés
    max_seq_length=2048,
    per_device_train_batch_size=hyperparameters["batch_size"],
    learning_rate=hyperparameters["learning_rate"],
    num_train_epochs=hyperparameters["num_train_epochs"],
    dataset_text_field="text", # Le nom de la colonne contenant le texte dans votre dataset
    # Pour la visualisation avec TensorBoard
    report_to=["tensorboard"],
    logging_dir="./logs", # Dossier pour les logs de TensorBoard
    logging_steps=10,
)

print("✅ Configurations créées !")
print("LoraConfig:", lora_config)
print("\nSFTConfig:", sft_config)

/Users/krst/Documents/Albert_school/LLM_Ops/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Configurations créées !
LoraConfig: LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='CAUSAL_LM', inference_mode=False, r=16, target_modules={'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'q_proj'}, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

SFTConfig: SFTConfig(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, prepare_model_for_kbit_training

# Nom du modèle sur le Hub Hugging Face
model_name = "microsoft/Phi-3-mini-4k-instruct"

# Charger le tokenizer et le modèle
# ⚠️ Attention : Sur Mac, spécifiez torch_dtype=torch.float32 si vous n'avez pas de GPU.
# Le chargement peut prendre du temps et beaucoup de RAM.
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto", # Laisse 'accelerate' gérer où placer le modèle (CPU/MPS)
    torch_dtype="auto" # Utilise le type de donnée le plus approprié
)

# ⚠️ La ligne suivante ne fonctionnera PAS sur Mac car elle dépend de bitsandbytes.
# Il faudra la remettre dans votre composant Kubeflow mais la commenter localement.
# model = prepare_model_for_kbit_training(model)

# Appliquer la configuration LoRA au modèle
model = get_peft_model(model, lora_config)

# Vérifier le nombre de paramètres entraînables
def print_trainable_parameters(model):
    """Affiche le nombre de paramètres entraînables dans le modèle."""
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable params: {trainable_params} || All params: {all_param} || "
        f"Trainable %: {100 * trainable_params / all_param:.2f}"
    )

print_trainable_parameters(model)

# Vous pouvez aussi afficher la structure du modèle pour voir les adaptateurs LoRA
# print(model)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.98s/it]
Some parameters are on the meta device because they were offloaded to the disk.


Trainable params: 8912896 || All params: 3829992448 || Trainable %: 0.23


In [5]:
import pandas as pd
from datasets import Dataset

# --- Charger les données depuis GCS et les formater ---

# L'URI de votre fichier
gcs_path = "gs://bucket-llm-ops/yoda_sentences.csv"

print(f"Chargement et formatage du dataset depuis {gcs_path}...")

try:
    # 1. Charger le fichier CSV dans un DataFrame pandas
    df = pd.read_csv(gcs_path)

    # 2. Créer la colonne 'text' en combinant 'sentence' et 'translation'
    #    avec le format de chat attendu par le modèle.
    df['text'] = (
        "<|user|>\n" +
        df['sentence'].astype(str) +
        "<|end|>\n<|assistant|>\n" +
        df['translation'].astype(str)
    )

    # 3. Garder uniquement la colonne 'text' qui est nécessaire pour l'entraînement
    df_formatted = df[['text']]

    # 4. Convertir le DataFrame pandas en objet Dataset de Hugging Face
    hf_dataset = Dataset.from_pandas(df_formatted)

    # 5. Diviser le jeu de données en entraînement et validation
    split_dataset = hf_dataset.train_test_split(test_size=0.1, seed=42)
    train_dataset = split_dataset["train"]
    eval_dataset = split_dataset["test"]

    print("✅ Données chargées et formatées avec succès !")
    print("\n---------- Aperçu du premier exemple formaté ----------")
    print(train_dataset[0]['text'])
    print("----------------------------------------------------")


except Exception as e:
    print(f"❌ Une erreur est survenue : {e}")
    print("Vérifiez que vous êtes bien authentifié ('gcloud auth application-default login') et que les bibliothèques sont installées.")

Chargement et formatage du dataset depuis gs://bucket-llm-ops/yoda_sentences.csv...
✅ Données chargées et formatées avec succès !

---------- Aperçu du premier exemple formaté ----------
<|user|>
Ten pins were set in order.<|end|>
<|assistant|>
In order, ten pins were set.
----------------------------------------------------


In [7]:
from trl import SFTTrainer

# Code qui FONCTIONNE
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=lora_config,
)

# Lancer l'entraînement
# ⚠️ ATTENTION : Ceci peut être très lent sur un Mac. Le but est juste de
# vérifier que ça ne crash pas au démarrage. Vous pouvez l'arrêter après quelques étapes.
print("🚀 Lancement de l'entraînement de test...")
trainer.train()
print("✅ Entraînement de test terminé !")

Truncating eval dataset: 100%|██████████| 72/72 [00:00<00:00, 70443.17 examples/s]


NotImplementedError: Cannot copy out of meta tensor; no data! Please use torch.nn.Module.to_empty() instead of torch.nn.Module.to() when moving module from meta to a different device.

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model

# Nom du modèle
model_name = "microsoft/Phi-3-mini-4k-instruct"

# --- CODE MODIFIÉ ICI ---

# 1. Détecter le meilleur appareil disponible sur Mac (MPS pour Apple Silicon, sinon CPU)
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Utilisation de l'appareil : {device}")

# 2. Charger le tokenizer (inchangé)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 3. Charger le modèle en forçant l'appareil
#    On remplace device_map="auto" par device_map={"": device}
#    Cela dit à 'accelerate' de mettre le modèle ENTIER ("") sur l'appareil détecté.
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map={"": device}, # <--- LA MODIFICATION CLÉ
    torch_dtype="auto",
)

# --- FIN DU CODE MODIFIÉ ---

# La suite est inchangée
# (On commente la ligne qui ne fonctionne pas sur Mac)
# model = prepare_model_for_kbit_training(model)

model = get_peft_model(model, lora_config)

print_trainable_parameters(model)

Utilisation de l'appareil : mps


Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.00s/it]


Trainable params: 8912896 || All params: 3829992448 || Trainable %: 0.23


In [9]:
import pandas as pd
from datasets import Dataset

# --- Charger les données depuis GCS et les formater ---

# L'URI de votre fichier
gcs_path = "gs://bucket-llm-ops/yoda_sentences.csv"

print(f"Chargement et formatage du dataset depuis {gcs_path}...")

try:
    # 1. Charger le fichier CSV dans un DataFrame pandas
    df = pd.read_csv(gcs_path)

    # 2. Créer la colonne 'text' en combinant 'sentence' et 'translation'
    #    avec le format de chat attendu par le modèle.
    df['text'] = (
        "<|user|>\n" +
        df['sentence'].astype(str) +
        "<|end|>\n<|assistant|>\n" +
        df['translation'].astype(str)
    )

    # 3. Garder uniquement la colonne 'text' qui est nécessaire pour l'entraînement
    df_formatted = df[['text']]

    # 4. Convertir le DataFrame pandas en objet Dataset de Hugging Face
    hf_dataset = Dataset.from_pandas(df_formatted)

    # 5. Diviser le jeu de données en entraînement et validation
    split_dataset = hf_dataset.train_test_split(test_size=0.1, seed=42)
    train_dataset = split_dataset["train"]
    eval_dataset = split_dataset["test"]

    print("✅ Données chargées et formatées avec succès !")
    print("\n---------- Aperçu du premier exemple formaté ----------")
    print(train_dataset[0]['text'])
    print("----------------------------------------------------")


except Exception as e:
    print(f"❌ Une erreur est survenue : {e}")
    print("Vérifiez que vous êtes bien authentifié ('gcloud auth application-default login') et que les bibliothèques sont installées.")

Chargement et formatage du dataset depuis gs://bucket-llm-ops/yoda_sentences.csv...
✅ Données chargées et formatées avec succès !

---------- Aperçu du premier exemple formaté ----------
<|user|>
Ten pins were set in order.<|end|>
<|assistant|>
In order, ten pins were set.
----------------------------------------------------


In [10]:
from trl import SFTTrainer

# Code qui FONCTIONNE
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=lora_config,
)

# Lancer l'entraînement
# ⚠️ ATTENTION : Ceci peut être très lent sur un Mac. Le but est juste de
# vérifier que ça ne crash pas au démarrage. Vous pouvez l'arrêter après quelques étapes.
print("🚀 Lancement de l'entraînement de test...")
trainer.train()
print("✅ Entraînement de test terminé !")

Truncating eval dataset: 100%|██████████| 72/72 [00:00<00:00, 68900.27 examples/s]


🚀 Lancement de l'entraînement de test...


  0%|          | 0/648 [00:00<?, ?it/s]/Users/krst/Documents/Albert_school/LLM_Ops/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
You are not running the flash-attention implementation, expect numerical differences.
  2%|▏         | 10/648 [00:27<08:52,  1.20it/s] 

{'loss': 3.2715, 'grad_norm': 7.364518165588379, 'learning_rate': 0.0001969135802469136, 'num_tokens': 254.0, 'mean_token_accuracy': 0.4290195554494858, 'epoch': 0.02}


  3%|▎         | 20/648 [00:54<32:04,  3.07s/it]  

...

{'loss': 1.4043, 'grad_norm': 3.3530843257904053, 'learning_rate': 0.0001814814814814815, 'num_tokens': 1599.0, 'mean_token_accuracy': 0.7256359279155731, 'epoch': 0.09}


 11%|█         | 70/648 [10:27<7:05:00, 44.12s/it]

{'loss': 1.5786, 'grad_norm': 1.8639776706695557, 'learning_rate': 0.00017839506172839508, 'num_tokens': 1885.0, 'mean_token_accuracy': 0.71882643699646, 'epoch': 0.11}


 11%|█         | 71/648 [10:41<5:36:26, 34.98s/it]

KeyboardInterrupt: 